# 附录10.2 使用langsmith做prompt调用的跟踪


## 1. 安装依赖包

In [ ]:
!pip install -U langsmith openai

## 2. 创建Langsmith API Key

访问页面Langsmith并创建API KEY [https://smith.langchain.com/o/64cfaf30-2541-510a-9884-b3fd89509d24/settings/apikeys]. 

## 3. 设置环境变量

在环境变量里设置LANGSMITH_TRACING和LANGSMITH_API_KEY，或直接使用下面代码引用



In [ ]:
# export LANGSMITH_TRACING=true
# export LANGSMITH_API_KEY="<your-langsmith-api-key>"
# The example uses OpenAI, but it's not necessary if your code uses another LLM provider
# export OPENAI_API_KEY="<your-openai-api-key>"

# 读取LANGSMITH_API_KEY
import os
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')
if langsmith_api_key:
    print(f"LANGSMITH_API_KEY: {langsmith_api_key[:10]}...{langsmith_api_key[-4:] if len(langsmith_api_key) > 14 else '***'}")
else:
    print("LANGSMITH_API_KEY 未设置")

# 读取LANGSMITH_TRACING
langsmith_tracing = os.getenv('LANGSMITH_TRACING')
if langsmith_tracing:
    print(f"LANGSMITH_TRACING: {langsmith_tracing}")
else:
    print("LANGSMITH_TRACING 未设置")


LANGSMITH_API_KEY: lsv2_pt_82...c6ce
LANGSMITH_TRACING: true


## 4. 使用langchain的wraper来保证所有的大模型的调用被跟踪

In [5]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(OpenAI())

# This is the retriever we will use in RAG
# This is mocked out, but it could be anything we want
def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results

# This is the end-to-end RAG chain.
# It does a retrieval step then calls OpenAI
def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:
    
    {docs}""".format(docs="\n".join(docs))
    
    return openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )

In [6]:
rag("where did harrison work")

ChatCompletion(id='chatcmpl-BzDT5skOsbiOjgARjcotgc5y3mu2L', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Harrison worked at Kensho.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1753929479, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=7, prompt_tokens=33, total_tokens=40, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## 5. 查看调用结果[Link](https://smith.langchain.com/public/e7b7d256-10fe-4d49-a8d5-36ca8e5af0d2/r?_gl=1*1elnobj*_gcl_au*MTI5NDQwNTY2NC4xNzUwOTA5MTIx*_ga*MTUwMTA5NDE0OS4xNzUzODYzMzI3*_ga_47WX3HKKY2*czE3NTM5MjYzMDIkbzUkZzEkdDE3NTM5MjgwMjckajYwJGwwJGgw)




